In [2]:
import numpy as np
import matplotlib.pyplot as plt
from numba import jit
import os

In [3]:
#Constantes
G = 6.67e-11  # Constante de gravitación universal (m^3 kg^-1 s^-2)
c = 1.496e11  # Unidad astronómica (m)
h = 0.0001  # Paso de tiempo (s)
num_iter = 100000  # Número de iteraciones
num_planets = 9  # Número de planetas
skip = 400

# Masas de los planetas
Ms = 1.989e30  # Masa del Sol (kg)
Mmer = 3.302e23  # Masa de Mercurio (kg)
Mv = 4.867e24  # Masa de Venus (kg)
Mt = 5.972e24  # Masa de Tierra (kg)
Mmar = 6.417e23  # Masa de Marte (kg)
Mj = 1.898e27  # Masa de Júpiter (kg)
Msat = 5.683e26  # Masa de Saturno (kg)
Mu = 8.681e25  # Masa de Urano (kg)
Mn = 1.024e26  # Masa de Neptuno (kg)



#Masas reescaladas (M/Ms)
m = np.array(
    [[1],
    [Mmer/Ms],
    [Mv/Ms],
    [Mt/Ms],
    [Mmar/Ms],
    [Mj/Ms],
    [Msat/Ms],
    [Mu/Ms],
    [Mn/Ms]])

# Posiciones iniciales reescaladas (r/c)
r = np.array(
    [[0,1e-15],
    [57.9e9/c,1e-15],
    [108.2e9/c,1e-15],
    [1,1e-15],
    [227.9e9/c,1e-15],
    [778.6e9/c,1e-15],
    [1433.5e9/c,1e-15],
    [2872.5e9/c,1e-15],
    [4495.1e9/c,1e-15]])

# Velocidad de referencia (m/s)
v_r = np.sqrt(G * Ms/ c)

# Velocidades iniciales reescaladas (v/v_r)
v = np.array(
    [[0, 0],
    [0, 47872/v_r],
    [0, 35020/v_r],
    [0, 29780/v_r],
    [0, 24077/v_r],
    [0, 13070/v_r],
    [0, 9620/v_r],
    [0, 6810/v_r],
    [0, 5478/v_r]])


In [9]:
#Algoritmo de Verlet
@jit(nopython=True, fastmath=True)
def verlet(m,r,v, num_iter, num_planets,skip):
    # Inicializar arrays para almacenar posiciones, velocidades y aceleraciones
    pos = np.zeros((num_iter, num_planets, 2))
    vel = np.zeros((num_iter, num_planets, 2))
    per = np.ones(num_planets)

    a = np.zeros((num_planets, 2))
    w = np.zeros((num_planets, 2))

    r_new = np.zeros((num_planets, 2))
    v_new = np.zeros((num_planets, 2))
    a_new = np.zeros((num_planets, 2))

    for t in range(num_iter):

        # 0) Almacenar posiciones y veloci        for i in range(num_planets):
        if t%skip == 0:
            for i in range(num_planets):
                pos[int(t/skip),i] = r[i]
                vel[int(t/skip),i] = v[i]


        # 1) Evaluar la aceleración inicial
        for i in range(num_planets):
            aux1 = np.array([0.0,0.0])
            for j in range(num_planets):
                if i != j:
                    aux1 = aux1 - m[j] * (r[i] - r[j]) / np.linalg.norm(r[i] - r[j])**3
            a[i] = aux1

        # 2) Evaluar la posición en t+h
        for i in range(num_planets):
            w[i] = v[i] + 0.5 * h * a[i]

        for i in range(num_planets):
            r_new[i] = r[i] + h * w[i]

        # 3) Evaluar la aceleración en t+h
        for i in range(num_planets):
            aux2 = np.array([0.0,0.0])
            for j in range(num_planets):
                if i != j:
                    aux2 = aux2 - m[j] * (r_new[i] - r_new[j]) / np.linalg.norm(r_new[i] - r_new[j])**3
            a_new[i] = aux2
        
        # 4) Evaluar la velocidad en t+h
        for i in range(num_planets):
            v_new[i] = w[i] + 0.5 * h * (a[i] + a_new[i])

        # 5) Almacenar posiciones y velocidades
        for i in range(num_planets):
            r[i] = r_new[i]
            v[i] = v_new[i]
        
        # 6) Almacenar periodos
        for i in range(num_planets):
            if per[i] == 1 and r[i][1] < 0:
                per[i] = 2*t    #Consideramos t cuando el planeta pasa por el eje x, como semiperiodo

    return pos, vel, per


In [10]:
wd = os.path.dirname("D:/DOCUMENTOS/GRANADA/4/COMPU/JimenezdaSilva/Obligatorio_1")  # Directorio de trabajo
datosPath = os.path.join(wd,"posPlanetas1.dat")  
ficheroPlot = open(datosPath, "w")

# CÁLCULO DE POSICIONES, VELOCIDADES Y PERÍODOS MEDIANTE LA FUNCIÓN "verlet()"
r,v,T = verlet(m,r,v,num_iter,num_planets,skip)

for t in range(int(num_iter/skip-1)):

    # ESCRITURA EN FICHERO
    for p in range(num_planets):
        ficheroPlot.write(str(r[t,p,0]) + ", " + str(r[t,p,1]) + "\n")
    ficheroPlot.write("\n") 


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mCannot unify array(float64, 1d, C) and array(float64, 2d, C) for 'aux1.2', defined at C:\Users\Jorge\AppData\Local\Temp\ipykernel_14912\722032173.py (30)
[1m
File "C:\Users\Jorge\AppData\Local\Temp\ipykernel_14912\722032173.py", line 30:[0m
[1mdef verlet(m,r,v, num_iter, num_planets,skip):
    <source elided>
                if i != j:
[1m                    aux1 = aux1 - m[j] * (r[i] - r[j]) / np.linalg.norm(r[i] - r[j])**3
[0m                    [1m^[0m[0m
[0m
[0m[1mDuring: typing of assignment at C:\Users\Jorge\AppData\Local\Temp\ipykernel_14912\722032173.py (30)[0m
[1m
File "C:\Users\Jorge\AppData\Local\Temp\ipykernel_14912\722032173.py", line 30:[0m
[1mdef verlet(m,r,v, num_iter, num_planets,skip):
    <source elided>
                if i != j:
[1m                    aux1 = aux1 - m[j] * (r[i] - r[j]) / np.linalg.norm(r[i] - r[j])**3
[0m                    [1m^[0m[0m

[0m[1mDu